# Title
[]()

In [4]:

import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
from silvhua import *
from datetime import datetime
# sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")


In [13]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [4]:
from wrangling import filter_df_all_conditions, filter_df_any_condition

# Code from 2023-06-28 notebook

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain

def create_LLMChain(template, input_variables, output_key, llm=OpenAI(temperature=1, max_tokens=100)):
    """
    Create an LLMChain object.
    Parameters:
        - template (str): The template for the prompt, i.e. the value of the `template` 
            parameter in the PromptTemplate class.
        - input_variables (List[str]) 
        - output_key (str): The key of the output variable.
        - llm (OpenAI object): Default is OpenAI()

    Returns:
        - LLMChain object
    """
    prompt_template = PromptTemplate(
        template=template, input_variables=input_variables)
    chain = LLMChain(
        llm=llm, prompt=prompt_template, output_key=output_key)
    return chain, prompt_template.template

def analyze_text_sentiment(text, llm=OpenAI(temperature=0.7, max_tokens=200)):
    """
    Analyze the sentiment of a text.
    Parameters:
        - text (str): The text to analyze.
        - llm (OpenAI object): Default is OpenAI()

    Returns:
        - sentiment (str): The sentiment of the text.
    """
    
    # openai.api_key = os.getenv('api_openai_p')
    template_root = """
    You are the owner of a personal training business. You use a Facebook group to as a lead generation tool.
    You aim to respond quickly to posts by group members if they request information, \
    express a desire to improve their fitness, or express dissatisfaction with their current fitness.
    Given the following member post delimitted by triple backticks, 
    """
    templates = {
        'sentiment': 'determine the sentiment of the message as "positive", "neutral", or "negative".',
        'emotions': 'determine the emotions expressed in the message.',
        'respond': 'indicate whether you need to quickly respond to the message or not using "yes" or "no".',
    }
    chain_dict = {}
    prompt_dict = {}
    for template in templates:
        chain_dict[template], prompt_dict[template] = create_LLMChain(
            template_root+templates[template]+' ```{text}```', 
            input_variables=['text'], output_key=template,
            llm=llm)
    overall_chain = SequentialChain(
        chains = [chain for chain in chain_dict.values()], 
        input_variables = ['text' for chain in chain_dict.values()],
        output_variables = list(chain_dict.keys()), verbose=True
    )
    response_dict = overall_chain(text)
    return response_dict, prompt_dict

def batch_analyze_text_sentiment(messages_list):
    response_dict = dict()
    prompt_dict = dict()
    failures = 0
    for iteration_id, message in enumerate(messages_list):
        try:
            response_dict[iteration_id], prompt_dict[iteration_id] = analyze_text_sentiment(message)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            file = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", file, ":", error)
            print('\t...Error making chatbot request')
            failures += 1
            if failures > 2:
                print('\t...Too many failures, stopping')
                break
    return response_dict, prompt_dict

def create_df_from_response(response):
    """
    Convert the response into a dataframe and strip white spaces at the start and end of text.
    """
    df = pd.DataFrame(response).transpose()
    for column in df.columns:
        df[column] = df[column].str.strip().str.replace('\n', ' ')
    return df

iteration_id = 2
# prompt_dict = dict()
# message_dict = dict()

messages_list = [
    'I struggle to pick up my toddler',
    "I want my arms to look like Michelle Obama\'s",
    "What's the best way to get fit?",
    "My knees are always achy",
    "My personal trainer doesn't understand me",
    "I love your ball gown",
    "Nancy looks great in that photo",
    "Happy Mother's Day!",
    # "I am preparing to run a marathon.",
    # "I am preparing to run a marathon!",
    # "My doctor warned me about my cholesterol but I think he's stupid."
]



response[iteration_id], prompt_dict[iteration_id] = batch_analyze_text_sentiment(messages_list)
response[iteration_id]



> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


{0: {'text': 'I struggle to pick up my toddler',
  'sentiment': '\n\nNeutral',
  'emotions': '\n\nThe emotions expressed in the message are frustration and difficulty.',
  'respond': '\n\nYes'},
 1: {'text': "I want my arms to look like Michelle Obama's",
  'sentiment': '\n\nPositive',
  'emotions': '\n\nThe emotions expressed in the message are desire and admiration.',
  'respond': '\n\nYes'},
 2: {'text': "What's the best way to get fit?",
  'sentiment': '\n\nPositive',
  'emotions': '\n\nAnswer: Curiosity, Desire for Improvement',
  'respond': '\n\nYes.'},
 3: {'text': 'My knees are always achy',
  'sentiment': '\n\nNegative',
  'emotions': '\n\nThe emotions expressed in the message are pain and discomfort.',
  'respond': '\n\nYes'},
 4: {'text': "My personal trainer doesn't understand me",
  'sentiment': '\n\nNegative',
  'emotions': '\n\nThe emotions expressed in this message are frustration and dissatisfaction.',
  'respond': '\n\nYes'},
 5: {'text': 'I love your ball gown',
  's

# Load objects

In [ ]:
filename = 'FB_group_posts_and_comments_2023-06-30_1550.sav'
path = r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\ChatGPT-sentiment-analysis\outputs\raw'
df = loadpickle(filename, path)
df

In [43]:
df['parent_comment_message'].unique().shape

(62,)

In [44]:
df['post_message'].unique().shape

(30,)

# Set up

In [51]:
prompt_dict = dict()
message_dict = dict()
response_dict = dict()

# Update scripts

## Iteration 1

In [27]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
import os
os.getenv('OPENAI_API_KEY')

def process_posts(df):
    """
    Return the unique non-null parent_comment_message values and the associated post_message values.
    """
    df = df[['parent_comment_message', 'post_message']].drop_duplicates()
    df = df[df['parent_comment_message'].notnull()]
    df = df[df['post_message'].notnull()]
    # results = pd.Series(df.apply(lambda row: f"You: `{row['post_message']}`\n\nMember: `{row['parent_comment_message']}`", axis=1).unique())
    results = pd.DataFrame(
        df.apply(lambda row: f"You: {row['post_message']}\n\nMember: {row['parent_comment_message']}", axis=1).unique(),
        columns=['text'])
    
    results['text'] = results['text'].str.replace('Amanda', 'Amy')
    results['text'] = results['text'].str.replace('amanda', 'amy')
    return results

def create_LLMChain(template, input_variables, output_key, llm=OpenAI(temperature=0, max_tokens=100)):
    """
    Create an LLMChain object.
    Parameters:
        - template (str): The template for the prompt, i.e. the value of the `template` 
            parameter in the PromptTemplate class.
        - input_variables (List[str]) 
        - output_key (str): The key of the output variable.
        - llm (OpenAI object): Default is OpenAI()

    Returns:
        - LLMChain object
    """
    prompt_template = PromptTemplate(
        template=template, input_variables=input_variables)
    chain = LLMChain(
        llm=llm, prompt=prompt_template, output_key=output_key)
    return chain, prompt_template.template

def analyze_text_sentiment(text, llm=OpenAI(temperature=0, max_tokens=200)):
    """
    Analyze the sentiment of a text.
    Parameters:
        - text (str): The text to analyze.
        - llm (OpenAI object): Default is OpenAI()

    Returns:
        - sentiment (str): The sentiment of the text.
    """
    
    # openai.api_key = os.getenv('api_openai_p')
    template_root = """
    You are the owner of a personal training business. You use a Facebook group to as a lead generation tool.
    You aim to respond quickly to posts by group members if they request information, \
    express a desire to improve their fitness, or express dissatisfaction with their current fitness.
    Given the following member post delimitted by triple backticks, 
    """
    templates = {
        'sentiment': 'determine the sentiment of the message as "positive", "neutral", or "negative".',
        'emotions': 'determine the emotions expressed in the message.',
        'respond': 'indicate whether you need to quickly respond to the message or not using "yes" or "no".',
    }
    chain_dict = {}
    prompt_dict = {}
    for template in templates:
        chain_dict[template], prompt_dict[template] = create_LLMChain(
            template_root+templates[template]+' ```{text}```', 
            input_variables=['text'], output_key=template,
            llm=llm)
    overall_chain = SequentialChain(
        chains = [chain for chain in chain_dict.values()], 
        input_variables = ['text' for chain in chain_dict.values()],
        output_variables = list(chain_dict.keys()), verbose=True
    )
    response_dict = overall_chain(text)
    return response_dict, prompt_dict

def batch_analyze_text_sentiment(messages_list):
    response_dict = dict()
    prompt_dict = dict()
    failures = 0
    for iteration_id, message in enumerate(messages_list):
        try:
            response_dict[iteration_id], prompt_dict[iteration_id] = analyze_text_sentiment(message)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            file = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", file, ":", error)
            print('\t...Error making chatbot request')
            failures += 1
            if failures > 2:
                print('\t...Too many failures, stopping')
                break
    return response_dict, prompt_dict

def create_df_from_response(response):
    """
    Convert the response into a dataframe and strip white spaces at the start and end of text.
    """
    df = pd.DataFrame(response).transpose()
    for column in df.columns:
        df[column] = df[column].str.strip().str.replace('\n', ' ')
    return df

iteration_id = 1

process_posts(df)



# response[iteration_id], prompt_dict[iteration_id] = batch_analyze_text_sentiment(messages_list)
# response[iteration_id]

,text
0,You: Greetings team from Airlie Beach 👋\n\nReminder there will be no weekly live training next week as I am on holidays 🙏\n\nI’m curious - have you any holidays planned this year?\n\nMember: Yes 😁
1,You: Greetings team from Airlie Beach 👋\n\nReminder there will be no weekly live training next week as I am on holidays 🙏\n\nI’m curious - have you any holidays planned this year?\n\nMember: I’m in Airlie beach too!!
2,You: Greetings team from Airlie Beach 👋\n\nReminder there will be no weekly live training next week as I am on holidays 🙏\n\nI’m curious - have you any holidays planned this year?\n\nMember: You are less than 2 hours away from where I am thats so cool I love airlie beach just never find the time to go there anymore
3,"You: How I stay bikini confident year round 👙\n\nFirst off I need to tell you - being bikini confident has actually NOTHING to do with how you look and everything to do with how you feel.\n\nSo let me tell you a little story…\n\nOnce upon a time I was a self conscious, controlling perfectionist who believed that being skinny and lean would make me happy. \n\nI thought having abs and being lean were the definition of the perfect body.\n\nAnd I embarked on a journey to achieve ‘perfection’.\n\nI ended up exhausted, overwhelmed and disappointed 😔 \n\nI was unhappy - in a lot of elements of my life - so I decided to seek the help of a life coach. This was hard for me - as I’m a very strong independent woman so asking for help wasn’t easy. But let me tell you - hands down this changed my life.\n\nAfter a very difficult time in my life, I chose to work with a counsellor (for a TOTALLY different reason). \n\nSo what has this got to do with me being happy in a bikini? 🤣\n\nSeeking help from these people - changed my whole outlook on life. They helped me look internally and forced me to uncover some things I didn’t even know about myself. \n\nMy mindset towards myself changed. I stopped focusing on aesthetics as a goal and started working on performance. My relationship with food did a 360 and I started viewing food as fuel instead of calories in versus calories out. I started practising gratitude and meditating. I prioritised sleep instead of ‘grinding’. \n\nAnd (for anyone who read the whole copy) that is how I am now bikini confident year round 👙\n\nMember: …. And you look awesome 👏👏"
4,"You: How I stay bikini confident year round 👙\n\nFirst off I need to tell you - being bikini confident has actually NOTHING to do with how you look and everything to do with how you feel.\n\nSo let me tell you a little story…\n\nOnce upon a time I was a self conscious, controlling perfectionist who believed that being skinny and lean would make me happy. \n\nI thought having abs and being lean were the definition of the perfect body.\n\nAnd I embarked on a journey to achieve ‘perfection’.\n\nI ended up exhausted, overwhelmed and disappointed 😔 \n\nI was unhappy - in a lot of elements of my life - so I decided to seek the help of a life coach. This was hard for me - as I’m a very strong independent woman so asking for help wasn’t easy. But let me tell you - hands down this changed my life.\n\nAfter a very difficult time in my life, I chose to work with a counsellor (for a TOTALLY different reason). \n\nSo what has this got to do with me being happy in a bikini? 🤣\n\nSeeking help from these people - changed my whole outlook on life. They helped me look internally and forced me to uncover some things I didn’t even know about myself. \n\nMy mindset towards myself changed. I stopped focusing on aesthetics as a goal and started working on performance. My relationship with food did a 360 and I started viewing food as fuel instead of calories in versus calories out. I started practising gratitude and meditating. I prioritised sleep instead of ‘grinding’. \n\nAnd (for anyone who read the whole copy) that is how I am now bikini confident year round 👙\n\nMember: Mind set is everything! Recently in Thailand, our last day. 12 hrs 

In [28]:
process_posts(df)

,text
0,You: Greetings team from Airlie Beach 👋\n\nReminder there will be no weekly live training next week as I am on holidays 🙏\n\nI’m curious - have you any holidays planned this year?\n\nMember: Yes 😁
1,You: Greetings team from Airlie Beach 👋\n\nReminder there will be no weekly live training next week as I am on holidays 🙏\n\nI’m curious - have you any holidays planned this year?\n\nMember: I’m in Airlie beach too!!
2,You: Greetings team from Airlie Beach 👋\n\nReminder there will be no weekly live training next week as I am on holidays 🙏\n\nI’m curious - have you any holidays planned this year?\n\nMember: You are less than 2 hours away from where I am thats so cool I love airlie beach just never find the time to go there anymore
3,"You: How I stay bikini confident year round 👙\n\nFirst off I need to tell you - being bikini confident has actually NOTHING to do with how you look and everything to do with how you feel.\n\nSo let me tell you a little story…\n\nOnce upon a time I was a self conscious, controlling perfectionist who believed that being skinny and lean would make me happy. \n\nI thought having abs and being lean were the definition of the perfect body.\n\nAnd I embarked on a journey to achieve ‘perfection’.\n\nI ended up exhausted, overwhelmed and disappointed 😔 \n\nI was unhappy - in a lot of elements of my life - so I decided to seek the help of a life coach. This was hard for me - as I’m a very strong independent woman so asking for help wasn’t easy. But let me tell you - hands down this changed my life.\n\nAfter a very difficult time in my life, I chose to work with a counsellor (for a TOTALLY different reason). \n\nSo what has this got to do with me being happy in a bikini? 🤣\n\nSeeking help from these people - changed my whole outlook on life. They helped me look internally and forced me to uncover some things I didn’t even know about myself. \n\nMy mindset towards myself changed. I stopped focusing on aesthetics as a goal and started working on performance. My relationship with food did a 360 and I started viewing food as fuel instead of calories in versus calories out. I started practising gratitude and meditating. I prioritised sleep instead of ‘grinding’. \n\nAnd (for anyone who read the whole copy) that is how I am now bikini confident year round 👙\n\nMember: …. And you look awesome 👏👏"
4,"You: How I stay bikini confident year round 👙\n\nFirst off I need to tell you - being bikini confident has actually NOTHING to do with how you look and everything to do with how you feel.\n\nSo let me tell you a little story…\n\nOnce upon a time I was a self conscious, controlling perfectionist who believed that being skinny and lean would make me happy. \n\nI thought having abs and being lean were the definition of the perfect body.\n\nAnd I embarked on a journey to achieve ‘perfection’.\n\nI ended up exhausted, overwhelmed and disappointed 😔 \n\nI was unhappy - in a lot of elements of my life - so I decided to seek the help of a life coach. This was hard for me - as I’m a very strong independent woman so asking for help wasn’t easy. But let me tell you - hands down this changed my life.\n\nAfter a very difficult time in my life, I chose to work with a counsellor (for a TOTALLY different reason). \n\nSo what has this got to do with me being happy in a bikini? 🤣\n\nSeeking help from these people - changed my whole outlook on life. They helped me look internally and forced me to uncover some things I didn’t even know about myself. \n\nMy mindset towards myself changed. I stopped focusing on aesthetics as a goal and started working on performance. My relationship with food did a 360 and I started viewing food as fuel instead of calories in versus calories out. I started practising gratitude and meditating. I prioritised sleep instead of ‘grinding’. \n\nAnd (for anyone who read the whole copy) that is how I am now bikini confident year round 👙\n\nMember: Mind set is everything! Recently in Thailand, our last day. 12 hrs 

## 1.1

In [52]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
import os
os.getenv('OPENAI_API_KEY')

def process_posts(df_raw):
    """
    Return the unique non-null parent_comment_message values and the associated post_message values.
    """
    df = df_raw.copy()
    df = df[['parent_comment_message', 'post_message']].drop_duplicates()
    df = df[df['parent_comment_message'].notnull()]
    df = df[df['post_message'].notnull()]

    # Sort the dataframe by 'post_message' length in descending order
    df.sort_values(by='parent_comment_message', key=lambda x: x.str.len(), ascending=False, inplace=True)
    df.drop_duplicates(subset='post_message', keep='first', inplace=True)

    results = pd.DataFrame(
        df.apply(lambda row: f"You: {row['post_message']}\n\nMember: {row['parent_comment_message']}", axis=1).unique(),
        columns=['text'])
    
    results['text'] = results['text'].str.replace('Amanda', 'Amy')
    results['text'] = results['text'].str.replace('amanda', 'amy')
    print(f'Shape of processed DataFrame containing Facebook group posts and comments: {results.shape}')
    return results

def create_LLMChain(template, input_variables, output_key, llm=OpenAI(temperature=0, max_tokens=100)):
    """
    Create an LLMChain object.
    Parameters:
        - template (str): The template for the prompt, i.e. the value of the `template` 
            parameter in the PromptTemplate class.
        - input_variables (List[str]) 
        - output_key (str): The key of the output variable.
        - llm (OpenAI object): Default is OpenAI()

    Returns:
        - LLMChain object
    """
    prompt_template = PromptTemplate(
        template=template, input_variables=input_variables)
    chain = LLMChain(
        llm=llm, prompt=prompt_template, output_key=output_key)
    return chain, prompt_template.template

def analyze_text_sentiment(text, llm=OpenAI(temperature=0, max_tokens=200)):
    """
    Analyze the sentiment of a text.
    Parameters:
        - text (str): The text to analyze.
        - llm (OpenAI object): Default is OpenAI()

    Returns:
        - sentiment (str): The sentiment of the text.
    """
    
    # openai.api_key = os.getenv('api_openai_p')
    template_root = """
    You are the owner of a personal training business. You use a Facebook group to as a lead generation tool.
    You write posts in the group to engage with members and position yourself as \
    a coach who can help them achieve their fitness goals.
    You aim to respond quickly to comments by group members if they request information, \
    express a desire to improve their fitness, or express dissatisfaction with their current fitness.
    Below, delimited by triple back ticks, is a post you wrote in the group and a comment from a member:, 
    """
    templates = {
        'sentiment': 'Determine the sentiment of the member comment as "positive", "neutral", or "negative".',
        'emotions': 'Determine the main emotion expressed in the member comment.',
        'respond': 'Indicate whether you need to quickly respond to the message or not using "yes" or "no".',
    }
    chain_dict = {}
    prompt_dict = {}
    for template in templates:
        chain_dict[template], prompt_dict[template] = create_LLMChain(
            template_root+templates[template]+' ```{text}```', 
            input_variables=['text'], output_key=template,
            llm=llm)
    overall_chain = SequentialChain(
        chains = [chain for chain in chain_dict.values()], 
        input_variables = ['text' for chain in chain_dict.values()],
        output_variables = list(chain_dict.keys()), verbose=True
    )
    response_dict = overall_chain(text)
    return response_dict, prompt_dict

def batch_analyze_text_sentiment_from_df(messages_df, key='text'):
    response_dict = dict()
    prompt_dict = dict()
    messages_list = messages_df[key].tolist()
    failures = 0
    for iteration_id, message in enumerate(messages_list):
        try:
            response_dict[iteration_id], prompt_dict[iteration_id] = analyze_text_sentiment(message)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            file = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", file, ":", error)
            print('\t...Error making chatbot request')
            failures += 1
            if failures > 2:
                print('\t...Too many failures, stopping')
                break
    return response_dict, prompt_dict

def create_df_from_response(response):
    """
    Convert the response into a dataframe and strip white spaces at the start and end of text.
    """
    df = pd.DataFrame(response).transpose()
    for column in df.columns:
        df[column] = df[column].str.strip().str.replace('\n', ' ')
    return df

iteration_id = 1.1

input_df = process_posts(df)
input_df


Shape of processed DataFrame containing Facebook group posts and comments: (19, 1)


,text
0,"You: How I stay bikini confident year round 👙\n\nFirst off I need to tell you - being bikini confident has actually NOTHING to do with how you look and everything to do with how you feel.\n\nSo let me tell you a little story…\n\nOnce upon a time I was a self conscious, controlling perfectionist who believed that being skinny and lean would make me happy. \n\nI thought having abs and being lean were the definition of the perfect body.\n\nAnd I embarked on a journey to achieve ‘perfection’.\n\nI ended up exhausted, overwhelmed and disappointed 😔 \n\nI was unhappy - in a lot of elements of my life - so I decided to seek the help of a life coach. This was hard for me - as I’m a very strong independent woman so asking for help wasn’t easy. But let me tell you - hands down this changed my life.\n\nAfter a very difficult time in my life, I chose to work with a counsellor (for a TOTALLY different reason). \n\nSo what has this got to do with me being happy in a bikini? 🤣\n\nSeeking help from these people - changed my whole outlook on life. They helped me look internally and forced me to uncover some things I didn’t even know about myself. \n\nMy mindset towards myself changed. I stopped focusing on aesthetics as a goal and started working on performance. My relationship with food did a 360 and I started viewing food as fuel instead of calories in versus calories out. I started practising gratitude and meditating. I prioritised sleep instead of ‘grinding’. \n\nAnd (for anyone who read the whole copy) that is how I am now bikini confident year round 👙\n\nMember: Mind set is everything! Recently in Thailand, our last day. 12 hrs to kill before the flight, wanted to go for a swim, but didn’t want our baggage to weigh over the 25kg with wet swimmers (LOTS of shopping was done lol) so my friend and I stripped off to our bra and undies walked the 5 metres across the decking and went swimming! 😂🤣 12 mths ago before meeting u, I would never have dreamed of doing this 🤣😂🤣 you only have one body, just learn to love it! ❤️\nPS you look flipping amazballz!"
1,"You: Do it for you ❤️There will always be critics - silence them ✌️#beunapologeticallyyou\n\nMember: This comment makes me want to throat 👊🏻👊🏻👊🏻👊🏻\nI’m so glad you have broad shoulders, cause I think you look AMAZBALLZ!! I’d love to have a figure like yours, but I’m a wee bit busy, so I’ll settle for the one you helped me get!! ❤️😉 If someone is doing things to please another, what a shallow life they live! Love your response stay positive and keep smiling that gorgeous smile of yours! 💪🏻❤️🦵🏻"
2,"You: Hey @everyone Throw me some questions for our live training tomorrow? Nutrition, exercise, weight loss etc. anything you need to know - that’s what I’m here for\n\nMember: Need some tips for tightening tummy … lost 7kg in 3 months but now got stuck in same weight for more than 2 weeks … what do I need to change??"
3,You: @everyone can you please send me some questions for our live training Thursday! Last weeks questions were great! Remember I’m trying to help you all as much as possible so use this opportunity 👌\n\nMember: Did my measurements today and although my chest and hips seem to be coming down my stomach seems to be going up 😳 what am I doing wrong.
4,"You: Menopause - weights or cardio, 3 sessions a week - what to do, macros and calories - what changes, measurements\n\nMember: Thanks a million. You covered my question perfectly. I couldn't care less about calories but am trying to get the old bod healthy 🙂"
5,You: Greetings team from Airlie Beach 👋\n\nReminder there will be no weekly live training next week as I am on holidays 🙏\n\nI’m curious - have you any holidays planned this year?\n\nMember: You are less than 2 hours away from where I am thats so cool I love airlie beach just never find the time to go there anymore
6,"You: ‘I’ll never get results like her’\n\nDo you find yourself thinking or saying that? If you do - you’re not alone!!!!\n\nThis is something I hear REGULARL

In [54]:

response_dict[iteration_id], prompt_dict[iteration_id] = batch_analyze_text_sentiment(input_df)
response_dict[iteration_id]



> Entering new  chain...

> Finished chain.


{0: {'text': 'text',
  'sentiment': "\n\nPost:\nAre you ready to take your fitness to the next level?\n\nComment:\nYes, I'm ready to make a change!\n\nSentiment: Positive",
  'emotions': '\n\nThe main emotion expressed in the member comment is curiosity.',
  'respond': "\n\nPost:\nAre you ready to take your fitness to the next level?\n\nComment:\nYes, I'm ready to get started!\n\nYes"}}

In [55]:
create_df_from_response(response_dict[iteration_id])

,emotions,respond,sentiment,text
0,The main emotion expressed in the member comment is curiosity.,"Post: Are you ready to take your fitness to the next level? Comment: Yes, I'm ready to get started! Yes","Post: Are you ready to take your fitness to the next level? Comment: Yes, I'm ready to make a change! Sentiment: Positive",text


## 1.2 OpenAI API call

In [58]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
import os
os.getenv('OPENAI_API_KEY')

def process_posts(df_raw):
    """
    Return the unique non-null parent_comment_message values and the associated post_message values.
    """
    df = df_raw.copy()
    df = df[['parent_comment_message', 'post_message']].drop_duplicates()
    df = df[df['parent_comment_message'].notnull()]
    df = df[df['post_message'].notnull()]

    # Sort the dataframe by 'post_message' length in descending order
    df.sort_values(by='parent_comment_message', key=lambda x: x.str.len(), ascending=False, inplace=True)
    df.drop_duplicates(subset='post_message', keep='first', inplace=True)

    results = pd.DataFrame(
        df.apply(lambda row: f"You: {row['post_message']}\n\nMember: {row['parent_comment_message']}", axis=1).unique(),
        columns=['text'])
    
    results['text'] = results['text'].str.replace('Amanda', 'Amy')
    results['text'] = results['text'].str.replace('amanda', 'amy')
    print(f'Shape of processed DataFrame containing Facebook group posts and comments: {results.shape}')
    return results

def create_LLMChain(template, input_variables, output_key, llm=OpenAI(temperature=0, max_tokens=100)):
    """
    Create an LLMChain object.
    Parameters:
        - template (str): The template for the prompt, i.e. the value of the `template` 
            parameter in the PromptTemplate class.
        - input_variables (List[str]) 
        - output_key (str): The key of the output variable.
        - llm (OpenAI object): Default is OpenAI()

    Returns:
        - LLMChain object
    """
    prompt_template = PromptTemplate(
        template=template, input_variables=input_variables)
    chain = LLMChain(
        llm=llm, prompt=prompt_template, output_key=output_key)
    return chain, prompt_template.template

def analyze_text_sentiment(text, llm=OpenAI(temperature=0, max_tokens=200)):
    """
    Analyze the sentiment of a text.
    Parameters:
        - text (str): The text to analyze.
        - llm (OpenAI object): Default is OpenAI()

    Returns:
        - sentiment (str): The sentiment of the text.
    """
    
    # openai.api_key = os.getenv('api_openai_p')
    template_root = """
    You are the owner of a personal training business. You use a Facebook group to as a lead generation tool.
    You write posts in the group to engage with members and position yourself as \
    a coach who can help them achieve their fitness goals.
    You aim to respond quickly to comments by group members if they request information, \
    express a desire to improve their fitness, or express dissatisfaction with their current fitness.
    Below, delimited by triple back ticks, is a post you wrote in the group and a comment from a member:, 
    """
    templates = {
        'sentiment': 'Determine the sentiment of the member comment as "positive", "neutral", or "negative".',
        'emotions': 'Determine the main emotion expressed in the member comment.',
        'respond': 'Indicate whether you need to quickly respond to the message or not using "yes" or "no".',
    }
    chain_dict = {}
    prompt_dict = {}
    for template in templates:
        chain_dict[template], prompt_dict[template] = create_LLMChain(
            template_root+templates[template]+' ```{text}```', 
            input_variables=['text'], output_key=template,
            llm=llm)
    overall_chain = SequentialChain(
        chains = [chain for chain in chain_dict.values()], 
        input_variables = ['text' for chain in chain_dict.values()],
        output_variables = list(chain_dict.keys()), verbose=True
    )
    response_dict = overall_chain(text)
    return response_dict, prompt_dict

def batch_analyze_text_sentiment_from_df(messages_df, key='text'):
    response_dict = dict()
    prompt_dict = dict()
    messages_list = messages_df[key].tolist()
    failures = 0
    for iteration_id, message in enumerate(messages_list):
        # print(iteration_id, message)
        try:
            response_dict[iteration_id], prompt_dict[iteration_id] = analyze_text_sentiment(message)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            file = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", file, ":", error)
            print('\t...Error making chatbot request')
            failures += 1
            if failures > 2:
                print('\t...Too many failures, stopping')
                break
    return response_dict, prompt_dict

def create_df_from_response(response):
    """
    Convert the response into a dataframe and strip white spaces at the start and end of text.
    """
    df = pd.DataFrame(response).transpose()
    for column in df.columns:
        df[column] = df[column].str.strip().str.replace('\n', ' ')
    return df

iteration_id = 1.2

# input_df = process_posts(df)

response_dict[iteration_id], prompt_dict[iteration_id] = batch_analyze_text_sentiment_from_df(input_df)



> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


In [59]:
create_df_from_response(response_dict[iteration_id])

,text,sentiment,emotions,respond
0,"You: How I stay bikini confident year round 👙 First off I need to tell you - being bikini confident has actually NOTHING to do with how you look and everything to do with how you feel. So let me tell you a little story… Once upon a time I was a self conscious, controlling perfectionist who believed that being skinny and lean would make me happy. I thought having abs and being lean were the definition of the perfect body. And I embarked on a journey to achieve ‘perfection’. I ended up exhausted, overwhelmed and disappointed 😔 I was unhappy - in a lot of elements of my life - so I decided to seek the help of a life coach. This was hard for me - as I’m a very strong independent woman so asking for help wasn’t easy. But let me tell you - hands down this changed my life. After a very difficult time in my life, I chose to work with a counsellor (for a TOTALLY different reason). So what has this got to do with me being happy in a bikini? 🤣 Seeking help from these people - changed my whole outlook on life. They helped me look internally and forced me to uncover some things I didn’t even know about myself. My mindset towards myself changed. I stopped focusing on aesthetics as a goal and started working on performance. My relationship with food did a 360 and I started viewing food as fuel instead of calories in versus calories out. I started practising gratitude and meditating. I prioritised sleep instead of ‘grinding’. And (for anyone who read the whole copy) that is how I am now bikini confident year round 👙 Member: Mind set is everything! Recently in Thailand, our last day. 12 hrs to kill before the flight, wanted to go for a swim, but didn’t want our baggage to weigh over the 25kg with wet swimmers (LOTS of shopping was done lol) so my friend and I stripped off to our bra and undies walked the 5 metres across the decking and went swimming! 😂🤣 12 mths ago before meeting u, I would never have dreamed of doing this 🤣😂🤣 you only have one body, just learn to love it! ❤️ PS you look flipping amazballz!",Positive,The main emotion expressed in the member comment is positivity.,No
1,"You: Do it for you ❤️There will always be critics - silence them ✌️#beunapologeticallyyou Member: This comment makes me want to throat 👊🏻👊🏻👊🏻👊🏻 I’m so glad you have broad shoulders, cause I think you look AMAZBALLZ!! I’d love to have a figure like yours, but I’m a wee bit busy, so I’ll settle for the one you helped me get!! ❤️😉 If someone is doing things to please another, what a shallow life they live! Love your response stay positive and keep smiling that gorgeous smile of yours! 💪🏻❤️🦵🏻",Positive,The main emotion expressed in the member comment is admiration.,No
2,"You: Hey @everyone Throw me some questions for our live training tomorrow? Nutrition, exercise, weight loss etc. anything you need to know - that’s what I’m here for Member: Need some tips for tightening tummy … lost 7kg in 3 months but now got stuck in same weight for more than 2 weeks … what do I need to change??",Positive,The main emotion expressed in the member comment is frustration.,Yes
3,You: @everyone can you please send me some questions for our live training Thursday! Last weeks questions were great! Remember I’m trying to help you all as much as possible so use this opportunity 👌 Member: Did my measurements today and although my chest and hips seem to be coming down my stomach seems to be going up 😳 what am I doing wrong.,Negative,The main emotion expressed in the member comment is frustration.,Yes
4,"You: Menopause - weights or cardio, 3 sessions a week - what to do, macros and calories - what changes, measurements Member: Thanks a million. You covered my question perfectly. I couldn't care less about calories but am trying to get the old bod healthy 🙂",Positive,Gratitude,No
5,You: Greetings team from Airlie Beach 👋 Reminder there will be no weekly live training next week as I am on holidays 🙏 I’m curious - have you any holidays planned this year? Memb

## Iteration 2 Revise prompts 

In [60]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
import os
os.getenv('OPENAI_API_KEY')

def process_posts(df_raw):
    """
    Return the unique non-null parent_comment_message values and the associated post_message values.
    """
    df = df_raw.copy()
    df = df[['parent_comment_message', 'post_message']].drop_duplicates()
    df = df[df['parent_comment_message'].notnull()]
    df = df[df['post_message'].notnull()]

    # Sort the dataframe by 'post_message' length in descending order
    df.sort_values(by='parent_comment_message', key=lambda x: x.str.len(), ascending=False, inplace=True)
    df.drop_duplicates(subset='post_message', keep='first', inplace=True)

    results = pd.DataFrame(
        df.apply(lambda row: f"YOU: {row['post_message']}\n\nMEMBER: {row['parent_comment_message']}", axis=1).unique(),
        columns=['text'])
    
    results['text'] = results['text'].str.replace('Amanda', 'Amy')
    results['text'] = results['text'].str.replace('amanda', 'amy')
    print(f'Shape of processed DataFrame containing Facebook group posts and comments: {results.shape}')
    return results

def create_LLMChain(template, input_variables, output_key, llm=OpenAI(temperature=0, max_tokens=100)):
    """
    Create an LLMChain object.
    Parameters:
        - template (str): The template for the prompt, i.e. the value of the `template` 
            parameter in the PromptTemplate class.
        - input_variables (List[str]) 
        - output_key (str): The key of the output variable.
        - llm (OpenAI object): Default is OpenAI()

    Returns:
        - LLMChain object
    """
    prompt_template = PromptTemplate(
        template=template, input_variables=input_variables)
    chain = LLMChain(
        llm=llm, prompt=prompt_template, output_key=output_key)
    return chain, prompt_template.template

def analyze_text_sentiment(text, llm=OpenAI(temperature=0, max_tokens=200)):
    """
    Analyze the sentiment of a text.
    Parameters:
        - text (str): The text to analyze.
        - llm (OpenAI object): Default is OpenAI()

    Returns:
        - sentiment (str): The sentiment of the text.
    """
    
    # openai.api_key = os.getenv('api_openai_p')
    template_root = """
    You are the owner of a personal training business. You use a Facebook group to as a lead generation tool.
    You write posts in the group to engage with members and position yourself as \
    a coach who can help them achieve their fitness goals.
    You aim to respond quickly to comments by group members if they request information, \
    express a desire to improve their fitness, have health conditions or injuries, \
    or express dissatisfaction with their current fitness.
    Below, delimited by triple back ticks, is a post you wrote in the group and a comment from a member:, 
    """
    templates = {
        'sentiment': 'Determine the sentiment of the member comment as "positive", "neutral", or "negative".',
        'emotions': 'Determine the main emotion expressed in the member comment. Return your response as a single word.',
        'respond': 'Indicate whether you need to quickly respond to the message or not using "yes" or "no".',
    }
    template_end = 'If you are not sure of the answer, return "?"'
    chain_dict = {}
    prompt_dict = {}
    for template in templates:
        chain_dict[template], prompt_dict[template] = create_LLMChain(
            template_root+templates[template]+' ```{text}``` '+template_end, 
            input_variables=['text'], output_key=template,
            llm=llm)
    overall_chain = SequentialChain(
        chains = [chain for chain in chain_dict.values()], 
        input_variables = ['text' for chain in chain_dict.values()],
        output_variables = list(chain_dict.keys()), verbose=True
    )
    response_dict = overall_chain(text)
    return response_dict, prompt_dict

def batch_analyze_text_sentiment_from_df(messages_df, key='text'):
    response_dict = dict()
    prompt_dict = dict()
    messages_list = messages_df[key].tolist()
    failures = 0
    for iteration_id, message in enumerate(messages_list):
        # print(iteration_id, message)
        try:
            response_dict[iteration_id], prompt_dict[iteration_id] = analyze_text_sentiment(message)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            file = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", file, ":", error)
            print('\t...Error making chatbot request')
            failures += 1
            if failures > 2:
                print('\t...Too many failures, stopping')
                break
    return response_dict, prompt_dict

def create_df_from_response(response):
    """
    Convert the response into a dataframe and strip white spaces at the start and end of text.
    """
    df = pd.DataFrame(response).transpose()
    for column in df.columns:
        df[column] = df[column].str.strip().str.replace('\n', ' ')
    return df

iteration_id = 2

input_df = process_posts(df)

response_dict[iteration_id], prompt_dict[iteration_id] = batch_analyze_text_sentiment_from_df(input_df)

Shape of processed DataFrame containing Facebook group posts and comments: (19, 1)


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


In [61]:
create_df_from_response(response_dict[iteration_id])

,text,sentiment,emotions,respond
0,"YOU: How I stay bikini confident year round 👙 First off I need to tell you - being bikini confident has actually NOTHING to do with how you look and everything to do with how you feel. So let me tell you a little story… Once upon a time I was a self conscious, controlling perfectionist who believed that being skinny and lean would make me happy. I thought having abs and being lean were the definition of the perfect body. And I embarked on a journey to achieve ‘perfection’. I ended up exhausted, overwhelmed and disappointed 😔 I was unhappy - in a lot of elements of my life - so I decided to seek the help of a life coach. This was hard for me - as I’m a very strong independent woman so asking for help wasn’t easy. But let me tell you - hands down this changed my life. After a very difficult time in my life, I chose to work with a counsellor (for a TOTALLY different reason). So what has this got to do with me being happy in a bikini? 🤣 Seeking help from these people - changed my whole outlook on life. They helped me look internally and forced me to uncover some things I didn’t even know about myself. My mindset towards myself changed. I stopped focusing on aesthetics as a goal and started working on performance. My relationship with food did a 360 and I started viewing food as fuel instead of calories in versus calories out. I started practising gratitude and meditating. I prioritised sleep instead of ‘grinding’. And (for anyone who read the whole copy) that is how I am now bikini confident year round 👙 MEMBER: Mind set is everything! Recently in Thailand, our last day. 12 hrs to kill before the flight, wanted to go for a swim, but didn’t want our baggage to weigh over the 25kg with wet swimmers (LOTS of shopping was done lol) so my friend and I stripped off to our bra and undies walked the 5 metres across the decking and went swimming! 😂🤣 12 mths ago before meeting u, I would never have dreamed of doing this 🤣😂🤣 you only have one body, just learn to love it! ❤️ PS you look flipping amazballz!",Positive,Empathy,No
1,"YOU: Do it for you ❤️There will always be critics - silence them ✌️#beunapologeticallyyou MEMBER: This comment makes me want to throat 👊🏻👊🏻👊🏻👊🏻 I’m so glad you have broad shoulders, cause I think you look AMAZBALLZ!! I’d love to have a figure like yours, but I’m a wee bit busy, so I’ll settle for the one you helped me get!! ❤️😉 If someone is doing things to please another, what a shallow life they live! Love your response stay positive and keep smiling that gorgeous smile of yours! 💪🏻❤️🦵🏻",Positive,Encouragement,No
2,"YOU: Hey @everyone Throw me some questions for our live training tomorrow? Nutrition, exercise, weight loss etc. anything you need to know - that’s what I’m here for MEMBER: Need some tips for tightening tummy … lost 7kg in 3 months but now got stuck in same weight for more than 2 weeks … what do I need to change??",Positive,Desire,Yes
3,YOU: @everyone can you please send me some questions for our live training Thursday! Last weeks questions were great! Remember I’m trying to help you all as much as possible so use this opportunity 👌 MEMBER: Did my measurements today and although my chest and hips seem to be coming down my stomach seems to be going up 😳 what am I doing wrong.,Positive,Frustration,Yes
4,"YOU: Menopause - weights or cardio, 3 sessions a week - what to do, macros and calories - what changes, measurements MEMBER: Thanks a million. You covered my question perfectly. I couldn't care less about calories but am trying to get the old bod healthy 🙂",Positive,Thanks,No
5,YOU: Greetings team from Airlie Beach 👋 Reminder there will be no weekly live training next week as I am on holidays 🙏 I’m curious - have you any holidays planned this year? MEMBER: You are less than 2 hours away from where I am thats so cool I love airlie beach just never find the time to go there anymore,Positive,Disappointment,No
6,"YOU: ‘I’ll never get results like her’ Do you find yourself 

## 2.1 Process dataframe differently

In [64]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
import os
os.getenv('OPENAI_API_KEY')

def process_posts(df_raw):
    """
    Return the unique non-null parent_comment_message values and the associated post_message values.
    """
    df = df_raw.copy()
    df = df[['parent_comment_message', 'post_message']].drop_duplicates()
    df = df[df['parent_comment_message'].notnull()]
    df = df[df['post_message'].notnull()]

    # Sort the dataframe by 'post_message' length in descending order
    df.sort_values(by='parent_comment_message', key=lambda x: x.str.len(), ascending=False, inplace=True)
    df.drop_duplicates(subset='post_message', keep='first', inplace=True)

    results = pd.DataFrame(
        df.apply(lambda row: f"YOU: {row['post_message']}\n\nMEMBER: {row['parent_comment_message']}", axis=1).unique(),
        columns=['text'])
    
    results['text'] = results['text'].str.replace('Amanda', 'Amy')
    results['text'] = results['text'].str.replace('amanda', 'amy')
    print(f'Shape of processed DataFrame containing Facebook group posts and comments: {results.shape}')
    return results

def create_LLMChain(template, input_variables, output_key, llm=OpenAI(temperature=0, max_tokens=100)):
    """
    Create an LLMChain object.
    Parameters:
        - template (str): The template for the prompt, i.e. the value of the `template` 
            parameter in the PromptTemplate class.
        - input_variables (List[str]) 
        - output_key (str): The key of the output variable.
        - llm (OpenAI object): Default is OpenAI()

    Returns:
        - LLMChain object
    """
    prompt_template = PromptTemplate(
        template=template, input_variables=input_variables)
    chain = LLMChain(
        llm=llm, prompt=prompt_template, output_key=output_key)
    return chain, prompt_template.template

def analyze_text_sentiment(text, llm=OpenAI(temperature=0, max_tokens=200)):
    """
    Analyze the sentiment of a text.
    Parameters:
        - text (str): The text to analyze.
        - llm (OpenAI object): Default is OpenAI()

    Returns:
        - sentiment (str): The sentiment of the text.
    """
    
    # openai.api_key = os.getenv('api_openai_p')
    template_root = """
    You are the owner of a personal training business. You use a Facebook group to as a lead generation tool.
    You write posts in the group to engage with members and position yourself as \
    a coach who can help them achieve their fitness goals.
    You aim to respond quickly to comments by group members if they request information, \
    express a desire to improve their fitness, have health conditions or injuries, \
    or express dissatisfaction with their current fitness.
    Below, delimited by triple back ticks, is a post you wrote in the group and a comment from a member:, 
    """
    templates = {
        'sentiment': 'Determine the sentiment of the member comment as "positive", "neutral", or "negative".',
        'emotions': 'Determine the main emotion expressed in the member comment. Return your response as a single word.',
        'respond': 'Indicate whether you need to quickly respond to the message or not using "yes" or "no".',
    }
    template_end = 'If you are not sure of the answer, return "?"'
    chain_dict = {}
    prompt_dict = {}
    for template in templates:
        chain_dict[template], prompt_dict[template] = create_LLMChain(
            template_root+templates[template]+' ```{text}``` '+template_end, 
            input_variables=['text'], output_key=template,
            llm=llm)
    overall_chain = SequentialChain(
        chains = [chain for chain in chain_dict.values()], 
        input_variables = ['text' for chain in chain_dict.values()],
        output_variables = list(chain_dict.keys()), verbose=True
    )
    response_dict = overall_chain(text)
    return response_dict, prompt_dict

def batch_analyze_text_sentiment_from_df(messages_df, key='text'):
    response_dict = dict()
    prompt_dict = dict()
    messages_list = messages_df[key].tolist()
    failures = 0
    for iteration_id, message in enumerate(messages_list):
        # print(iteration_id, message)
        try:
            response_dict[iteration_id], prompt_dict[iteration_id] = analyze_text_sentiment(message)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            file = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", file, ":", error)
            print('\t...Error making chatbot request')
            failures += 1
            if failures > 2:
                print('\t...Too many failures, stopping')
                break
    return response_dict, prompt_dict

def create_df_from_response(response):
    """
    Convert the response into a dataframe and strip white spaces at the start and end of text.
    """
    df = pd.DataFrame(response).transpose()
    for column in df.columns:
        df[column] = df[column].str.strip().str.replace('\n', ' ')
        df[column] = df[column].str.strip().str.replace('YOU:', '[YOU]')
        df[column] = df[column].str.strip().str.replace('MEMBER:', '[MEMBER]')
    return df

iteration_id = 2

# input_df = process_posts(df)

# response_dict[iteration_id], prompt_dict[iteration_id] = batch_analyze_text_sentiment_from_df(input_df)

create_df_from_response(response_dict[iteration_id])

,text,sentiment,emotions,respond
0,"[YOU] How I stay bikini confident year round 👙 First off I need to tell you - being bikini confident has actually NOTHING to do with how you look and everything to do with how you feel. So let me tell you a little story… Once upon a time I was a self conscious, controlling perfectionist who believed that being skinny and lean would make me happy. I thought having abs and being lean were the definition of the perfect body. And I embarked on a journey to achieve ‘perfection’. I ended up exhausted, overwhelmed and disappointed 😔 I was unhappy - in a lot of elements of my life - so I decided to seek the help of a life coach. This was hard for me - as I’m a very strong independent woman so asking for help wasn’t easy. But let me tell you - hands down this changed my life. After a very difficult time in my life, I chose to work with a counsellor (for a TOTALLY different reason). So what has this got to do with me being happy in a bikini? 🤣 Seeking help from these people - changed my whole outlook on life. They helped me look internally and forced me to uncover some things I didn’t even know about myself. My mindset towards myself changed. I stopped focusing on aesthetics as a goal and started working on performance. My relationship with food did a 360 and I started viewing food as fuel instead of calories in versus calories out. I started practising gratitude and meditating. I prioritised sleep instead of ‘grinding’. And (for anyone who read the whole copy) that is how I am now bikini confident year round 👙 [MEMBER] Mind set is everything! Recently in Thailand, our last day. 12 hrs to kill before the flight, wanted to go for a swim, but didn’t want our baggage to weigh over the 25kg with wet swimmers (LOTS of shopping was done lol) so my friend and I stripped off to our bra and undies walked the 5 metres across the decking and went swimming! 😂🤣 12 mths ago before meeting u, I would never have dreamed of doing this 🤣😂🤣 you only have one body, just learn to love it! ❤️ PS you look flipping amazballz!",Positive,Empathy,No
1,"[YOU] Do it for you ❤️There will always be critics - silence them ✌️#beunapologeticallyyou [MEMBER] This comment makes me want to throat 👊🏻👊🏻👊🏻👊🏻 I’m so glad you have broad shoulders, cause I think you look AMAZBALLZ!! I’d love to have a figure like yours, but I’m a wee bit busy, so I’ll settle for the one you helped me get!! ❤️😉 If someone is doing things to please another, what a shallow life they live! Love your response stay positive and keep smiling that gorgeous smile of yours! 💪🏻❤️🦵🏻",Positive,Encouragement,No
2,"[YOU] Hey @everyone Throw me some questions for our live training tomorrow? Nutrition, exercise, weight loss etc. anything you need to know - that’s what I’m here for [MEMBER] Need some tips for tightening tummy … lost 7kg in 3 months but now got stuck in same weight for more than 2 weeks … what do I need to change??",Positive,Desire,Yes
3,[YOU] @everyone can you please send me some questions for our live training Thursday! Last weeks questions were great! Remember I’m trying to help you all as much as possible so use this opportunity 👌 [MEMBER] Did my measurements today and although my chest and hips seem to be coming down my stomach seems to be going up 😳 what am I doing wrong.,Positive,Frustration,Yes
4,"[YOU] Menopause - weights or cardio, 3 sessions a week - what to do, macros and calories - what changes, measurements [MEMBER] Thanks a million. You covered my question perfectly. I couldn't care less about calories but am trying to get the old bod healthy 🙂",Positive,Thanks,No
5,[YOU] Greetings team from Airlie Beach 👋 Reminder there will be no weekly live training next week as I am on holidays 🙏 I’m curious - have you any holidays planned this year? [MEMBER] You are less than 2 hours away from where I am thats so cool I love airlie beach just never find the time to go there anymore,Positive,Disappointment,No
6,"[YOU] ‘I’ll never get results like her’ Do you f

In [65]:
create_df_from_response(prompt_dict[iteration_id])

,sentiment,emotions,respond
0,"You are the owner of a personal training business. You use a Facebook group to as a lead generation tool. You write posts in the group to engage with members and position yourself as a coach who can help them achieve their fitness goals. You aim to respond quickly to comments by group members if they request information, express a desire to improve their fitness, have health conditions or injuries, or express dissatisfaction with their current fitness. Below, delimited by triple back ticks, is a post you wrote in the group and a comment from a member:, Determine the sentiment of the member comment as ""positive"", ""neutral"", or ""negative"". ```{text}``` If you are not sure of the answer, return ""?""","You are the owner of a personal training business. You use a Facebook group to as a lead generation tool. You write posts in the group to engage with members and position yourself as a coach who can help them achieve their fitness goals. You aim to respond quickly to comments by group members if they request information, express a desire to improve their fitness, have health conditions or injuries, or express dissatisfaction with their current fitness. Below, delimited by triple back ticks, is a post you wrote in the group and a comment from a member:, Determine the main emotion expressed in the member comment. Return your response as a single word. ```{text}``` If you are not sure of the answer, return ""?""","You are the owner of a personal training business. You use a Facebook group to as a lead generation tool. You write posts in the group to engage with members and position yourself as a coach who can help them achieve their fitness goals. You aim to respond quickly to comments by group members if they request information, express a desire to improve their fitness, have health conditions or injuries, or express dissatisfaction with their current fitness. Below, delimited by triple back ticks, is a post you wrote in the group and a comment from a member:, Indicate whether you need to quickly respond to the message or not using ""yes"" or ""no"". ```{text}``` If you are not sure of the answer, return ""?"""
1,"You are the owner of a personal training business. You use a Facebook group to as a lead generation tool. You write posts in the group to engage with members and position yourself as a coach who can help them achieve their fitness goals. You aim to respond quickly to comments by group members if they request information, express a desire to improve their fitness, have health conditions or injuries, or express dissatisfaction with their current fitness. Below, delimited by triple back ticks, is a post you wrote in the group and a comment from a member:, Determine the sentiment of the member comment as ""positive"", ""neutral"", or ""negative"". ```{text}``` If you are not sure of the answer, return ""?""","You are the owner of a personal training business. You use a Facebook group to as a lead generation tool. You write posts in the group to engage with members and position yourself as a coach who can help them achieve their fitness goals. You aim to respond quickly to comments by group members if they request information, express a desire to improve their fitness, have health conditions or injuries, or express dissatisfaction with their current fitness. Below, delimited by triple back ticks, is a post you wrote in the group and a comment from a member:, Determine the main emotion expressed in the member comment. Return your response as a single word. ```{text}``` If you are not sure of the answer, return ""?""","You are the owner of a personal training business. You use a Facebook group to as a lead generation tool. You write posts in the group to engage with members and position yourself as a coach who can help them achieve their fitness goals. You aim to respond quickly to comments by group members if they request information, express a desire to improve their fitness, have health con

# Save outputs

In [67]:
path = '../outputs/raw/2023-07-01'
to_save = {
    'prompt_dict': prompt_dict,
    'response_dict': response_dict,
}

for key, value in to_save.items():
    save_output(value, description=key, csv_path=None, pickle_path=path)

File saved:  ../outputs/raw/2023-07-01/prompt_dict_2023-07-02_1903.sav
	Time completed: 2023-07-02 19:03:39.183865
	Object saved as pickle
File saved:  ../outputs/raw/2023-07-01/response_dict_2023-07-02_1903.sav
	Time completed: 2023-07-02 19:03:39.201859
	Object saved as pickle


# *End of Page*